In [1]:
%env AWS_PROFILE=olek

env: AWS_PROFILE=olek


In [2]:
%pip install boto3
%pip install pandas
%pip install requests
%config Completer.use_jedi = False


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [62]:
from pathlib import Path
import csv
import pandas as pd
import requests
import base64
import boto3
from datetime import datetime
import os
import json

## Initial configuration

In [39]:
path_to_configuration = '../ignored/jupyter_configuration.csv'
config = pd.read_csv(path_to_configuration)
config['key']

0                                 api_path
1                             project_path
2                   path_to_bank_statement
3    step_functions_immediate_analysis_arn
4                                bank_type
Name: key, dtype: object

In [40]:
def get_value_from_config(key):
    return config.loc[config['key'] == key, 'value'].values[0]

In [41]:
def get_path_to_endpoint(endpoint):
    return get_value_from_config('api_path') + endpoint

In [42]:
path_to_bank_statement = get_value_from_config('path_to_bank_statement')
bank_type = get_value_from_config('bank_type')
step_functions_immediate_analysis_arn = get_value_from_config('step_functions_immediate_analysis_arn')
test_endpoint = get_path_to_endpoint('health')
post_bank_statement_endpoint = get_path_to_endpoint('bank-statements')

### Reading file content utils

In [43]:
text_encoding = 'Windows-1250'

def encode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64encode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

def decode_base64(text):
    text_bytes = text.encode(text_encoding)
    text_bytes_base64 = base64.b64decode(text_bytes)
    return text_bytes_base64.decode(text_encoding)

## Read bank statement

In [44]:
bank_statement_content = Path(path_to_bank_statement).read_text(encoding=text_encoding)

In [45]:
bank_statement_content_base64 = encode_base64(bank_statement_content)

## Upload bank statement to AWS

### Prepare filename

In [46]:
current_date = datetime.today().strftime('%Y-%m-%d')
bucket_name = 'expenses-counter-bank-statements-immediate-analysis'
filename = os.path.basename(path_to_bank_statement)
s3_filename = '{}/{}-{}'.format(current_date, bank_type, filename)
s3_filename

'2024-04-19/PKO_BP-pkobp.csv'

### Upload file to S3

In [22]:
s3 = boto3.resource('s3')

In [23]:
s3.meta.client.upload_file(path_to_bank_statement, bucket_name, s3_filename)

### Test connection

In [47]:
response = requests.get(test_endpoint)
response

<Response [403]>

### Call API Gateway for immediate analysis

In [122]:
api_gateway_body = {
    "input": "{}",# {"elo" : "melo"},
   "name": "MyExecution5",
   "stateMachineArn": step_functions_immediate_analysis_arn
}
api_gateway_body = json.dumps(api_gateway_body)

In [123]:
headers = {'content-type': 'application/json'}

In [120]:
response = requests.post(post_bank_statement_endpoint, headers=headers, data=str(api_gateway_body))
response.status_code

200

In [125]:
response.content

b'{"__type":"com.amazonaws.swf.service.v2.model#InvalidArn","message":"Invalid Arn: \'Resource type not valid in this context: execution\'"}'

In [ ]:
break

### Map bank statements to transactions

In [102]:
params = {
    "accountStatement": bank_statement_content_base64,
    "bankType": bank_type
}

In [103]:
headers = {'content-type': 'application/json'}
response = requests.get(parse_bank_statements_endpoint, params=params, headers=headers)
response.status_code

414